# Парсинг данных с карт

## Задача

В распоряжении есть датасет с адресами в Москве. На основе этих данных необходимо получить район и улицу

## Решение

### Подготовка

Импортируем библиотеки, с которыми будем работать

In [1]:
import pandas as pd
from yaml import load, FullLoader
import requests as req
import time

Считаем данные с адресами

In [2]:
path = 'datasets/'
file_name = 'rest_data.csv'

data = pd.read_csv(path + file_name)

Адреса в датасете имеют следующий вид

In [3]:
data['address'].sample(5)

2566               город Москва, улица Тёплый Стан, дом 1А
10396              город Москва, Онежская улица, дом 19/38
8260     город Москва, Верхняя Красносельская улица, до...
1031     город Москва, Садовая-Черногрязская улица, дом...
8679                 город Москва, Ярцевская улица, дом 19
Name: address, dtype: object

Уникальное кол-во адресов

In [4]:
len(data['address'].unique())

9108

Сохраним уникальные адреса в отдельный список

In [5]:
unique_adress = data['address'].unique().tolist()

### Решение задачи

Выполнить поставленную задачу можно разными путями. В этой работе я буду получать необходимые мне данные об адресе с помощью API <a href='https://tech.yandex.ru/maps/geocoder/?from=mapsapi' alt='Геокодер'>Геокодера</a> Яндекс.Карт. Для взаимодействия с сервисом необходимо зарегистрироваться и получить токен

In [6]:
config = load(open('config.yaml'), Loader=FullLoader)
token = config['yandex_maps_geocoder']['token']

Про формат запросов к API более подробно можно почитать <a href='https://tech.yandex.ru/maps/geocoder/doc/desc/concepts/input_params-docpage/' alt='Формат запросов к Геокодеру'>тут</a>. Сформируем ф-ию для отправки запросов <code>make_safe_request</code>

In [7]:
api_url = 'https://geocode-maps.yandex.ru/1.x'

In [8]:
# a simple request
def make_request(geocode):
    params = {
        'geocode':geocode,
        'format':'json',
        'apikey':token
    }
    
    response = req.get(api_url, params=params)
    return response

# requests with delay
def make_safe_request(geocode):
    response = []
    
    for n in range(3):
        try:
            response = make_request(geocode)
            break
        except:
            print('Error, wait {} seconds'.format(2**(n+1)))
            time.sleep(2**(n+1))
            
    return response

Алгоритм получения данных будет следующий:

1. Первым запросом необходимо получить координаты адреса, находящегося в <code>unique_adress</code>
2. Вторым запросом необходимо получить всю интересующую информацию по адресу с помощью координат, полученных на 1ом шаге

Такой формат сбора данных позволяет получить весь существующий набор данных об адресе, в т.ч. - район. Определим ф-ию получения координат

In [9]:
# for each address determines coordinates
def get_coordinates(address):
    result = []
    
    for geocode in address:
        response = make_safe_request(geocode)
        
        if response:
            response = response.json()
            
            try:
                coordinates = response['response']['GeoObjectCollection']\
                        ['featureMember'][0]['GeoObject']['Point']['pos']

                result.append([geocode, coordinates])
            except:
                result.append([geocode, 'Not_detected'])
        else:
            result.append([geocode, 'Not_detected'])
            
    return result

Получим координаты для адресов

In [10]:
coordinates = get_coordinates(unique_adress)

In [11]:
# Example of getting coordinates
coordinates[0]

['город Москва, улица Егора Абакумова, дом 9', '37.714474 55.879005']

Определим ф-ию <code>get_coordinates_info</code>, которая будет получать интересующие данные по координатам

In [12]:
# for each coordinates gets requried information
def get_coordinates_info(coordinates):
    result = []
    
    for geocode in coordinates:
        # change geocode format to "X,Y"
        response = make_safe_request(','.join(geocode.split()))
        
        if response:
            response = response.json()
            
            try:
                # 0 means 'precision': 'exact'
                address_det = response['response']['GeoObjectCollection']\
                        ['featureMember'][0]['GeoObject']\
                        ['metaDataProperty']['GeocoderMetaData']\
                        ['Address']['formatted']

                
                street = []

                # 1 means 'precision': 'street'
                for el in response['response']['GeoObjectCollection']\
                        ['featureMember'][1]['GeoObject']\
                        ['metaDataProperty']['GeocoderMetaData']\
                        ['Address']['Components']:

                    if el['kind'] == 'street':
                        street.append(el['name'])
                        

                district = []

                # 2 means 'precision': 'district'
                for el in response['response']['GeoObjectCollection']\
                        ['featureMember'][2]['GeoObject']\
                        ['metaDataProperty']['GeocoderMetaData']\
                        ['Address']['Components']:

                    if el['kind'] == 'district':
                        district.append(el['name'])
                
            
                result.append([geocode, address_det, street, district])
            except:
                result.append([geocode, 'Not_detected', 'Not_detected', 'Not_detected'])
        else:
            result.append([geocode, 'Not_detected', 'Not_detected', 'Not_detected'])
    
    return result

Не все координаты в результате 1ого шага могли быть получены. Выберем только уникальные значения (на всякий случай) и выбросим <code>'Not_detected'</code>

In [13]:
unique_coordinates = list(set([el[1] for el in coordinates]))

if 'Not_detected' in unique_coordinates:
    unique_coordinates.remove('Not_detected')

In [14]:
# Example of getting unique coordinates
unique_coordinates[0]

'37.549264 55.745924'

Наконец получим информацию по тем координатам, которые получили в 1ом шаге

In [16]:
coordinates_info = get_coordinates_info(unique_coordinates)

In [17]:
# Example of getting info by coordinates
coordinates_info[0]

['37.549264 55.745924',
 'Россия, Москва, Кутузовский проспект, 22',
 ['Кутузовский проспект'],
 ['Западный административный округ', 'район Дорогомилово']]

Создадим датафреймы <code>coordinates_df</code> и <code>coordinates_info_df</code> и добавим в них информацию из соответствующих списков

In [100]:
coordinates_df = pd.DataFrame(coordinates, columns=['address', 'coordinates'])

In [101]:
coordinates_info_df = pd.DataFrame(
    coordinates_info,
    columns=['coordinates', 'address_det', 'street', 'district']
)

Объединим датафреймы по <code>coordinates</code> в <code>address_info</code>

In [102]:
address_info = coordinates_df.merge(coordinates_info_df, on='coordinates', how='left')

In [103]:
address_info.head()

,address,coordinates,address_det,street,district
0,"город Москва, улица Егора Абакумова, дом 9",37.714474 55.879005,"Россия, Москва, улица Егора Абакумова, 9",[улица Егора Абакумова],"[Северо-Восточный административный округ, Ярос..."
1,"город Москва, улица Талалихина, дом 2/1, корпус 1",37.673295 55.738307,"Россия, Москва, улица Талалихина, 2/1к1",[улица Талалихина],"[Центральный административный округ, Таганский..."
2,"город Москва, Абельмановская улица, дом 6",37.669576 55.735571,"Россия, Москва, Абельмановская улица, 6",[Абельмановская улица],"[Центральный административный округ, Таганский..."
3,"город Москва, Абрамцевская улица, дом 1",37.573007 55.892713,"Россия, Москва, Абрамцевская улица, 1",[Абрамцевская улица],"[Северо-Восточный административный округ, райо..."
4,"город Москва, Абрамцевская улица, дом 9, корпус 1",37.572279 55.904074,"Россия, Москва, Абрамцевская улица, 9к1",[Абрамцевская улица],"[Северо-Восточный административный округ, райо..."


In [104]:
address_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9108 entries, 0 to 9107
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   address      9108 non-null   object
 1   coordinates  9108 non-null   object
 2   address_det  9083 non-null   object
 3   street       9083 non-null   object
 4   district     9083 non-null   object
dtypes: object(5)
memory usage: 426.9+ KB


### Доработка результатов

В результате предыдущих шагов были получены данные по интересующим нас адресам. Однако не для всех адресов координаты были определены:

In [105]:
address_info[address_info['coordinates'] == 'Not_detected']['coordinates'].count()

25

In [106]:
address_info[address_info['coordinates'] == 'Not_detected'].head()

,address,coordinates,address_det,street,district
4181,"город Москва, 87-й километр Московской Кольцев...",Not_detected,NaN,NaN,NaN
4217,"город Москва, 82-й километр Московской Кольцев...",Not_detected,NaN,NaN,NaN
4254,"город Москва, 87-й километр Московской Кольцев...",Not_detected,NaN,NaN,NaN
4394,"город Москва, 104-й километр Московской Кольце...",Not_detected,NaN,NaN,NaN
4798,"город Москва, 42-й километр Московской Кольцев...",Not_detected,NaN,NaN,NaN


Адреса, для которых не были получены координаты, все кроме одного находятся на МКАД. Рассмотрим внимательно исключение:

In [107]:
address_info.loc[6597, :]['address']

'город Москва, проспект Вернадского, дом 100, корпус ЗОНА, строение С-1'

По данному адресу находится военная академия вооруженных сил рф и, по всей видимости, координаты военных объектов Яндексу передавать запрещено. Однако, на портале <a href='https://data.mos.ru/opendata/7710881420-kombinaty-pitaniya/row/20730998' alt='data.mos.ru'>открытых данных Москвы</a> можно найти необходимую информацию об этом адресе. Вручную сформируем набор данных для этого адреса и добавим их в датафрейм

In [108]:
address_info.loc[6597, address_info.columns[1:]] = [
    '37.472772 55.649538',
    'Россия, Москва, проспект Вернадского, дом 100, корпус ЗОНА, строение С-1',
    ['проспект Вернадского'],
    ['Западный административный округ', 'район Тропарёво-Никулино']
]

Проверим результат

In [109]:
address_info.loc[6597, :]

address        город Москва, проспект Вернадского, дом 100, к...
coordinates                                  37.472772 55.649538
address_det    Россия, Москва, проспект Вернадского, дом 100,...
street                                    [проспект Вернадского]
district       [Западный административный округ, район Тропар...
Name: 6597, dtype: object

Для МКАДа заполним пропущенные значения болванкой "МКАД"

In [110]:
for index in address_info[address_info['coordinates'] == 'Not_detected'].index:
    address_info.loc[index, address_info.columns[2:]] = [
        'Россия, Москва, МКАД',
        ['МКАД'],
        ['МКАД'],
    ]

Осталось проверить <code>address_det</code>, <code>street</code>, <code>district</code> на <code>Not_detected</code>

In [111]:
for col in ['address_det', 'street', 'district']:
    print(address_info[address_info[col] == 'Not_detected'][col].count())

0
0
0


### Выводы

В результате работы с помощью Геокодера были получены интересующие данные относительно заданных адресов Москвы. Не для всех адресов были корректно получены улицы и районы Москвы. Связано это с тем, что в изначальных данных небольшое кол-во адресов были заданы без префикса "Москва," и необходимо было проверять получаемые от Геокодера данные на принадлежность городу Москва. Таких адресов действительно немного:

In [116]:
flag = 0

def moscow_det(elem):
    global flag
    if not (elem.find('Россия, Москва,') >= 0):
        flag = flag + 1

address_info['address_det'].apply(moscow_det)

flag

8

Также для некоторых адресов в <code>street</code> и <code>district</code> можно обнаружить '[]' - это означает, что у Яндекса нет иформации об улице или районе. Это не ошибка, а особенности местности. Например в Зеленограде не существует сущности "улица" и адреса имеют следующий вид: "Россия, Москва, Зеленоград, к1651".